In [1]:
import os
import re
import json, codecs
import pandas as pd
import numpy as np
from dateutil.parser import parse
from datetime import datetime, date, time, timedelta
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.preprocessing import scale
from sklearn import (manifold, decomposition, ensemble, discriminant_analysis, random_projection)
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [9]:
os.chdir('/project/')
sam = pd.read_csv('./Data/cli_cma_joined_1pct.csv', dtype=object)
# sam_cma = pd.read_csv('./Data/cli_cma_1pct.csv', dtype=object)
# sam_all = pd.merge(sam, sam_cma, on=['CREDITCYCLEFACTMKEY'])
sam.shape

(195343, 251)

In [10]:
sam.columns

Index([u'SNAPDATE', u'DWACCTID', u'CUSTOMERNUMBER', u'CYCLEDATE',
       u'HADINCREASE', u'FIRSTMEASURABLEDATE', u'MAXREVIEWDATE',
       u'INCREASEDATE', u'INCRTYPE', u'RANDOMNUM',
       ...
       u'CMA3926', u'CMA3932', u'CMA3151', u'CMA3635', u'CMA3170', u'CMA3209',
       u'CMA3660', u'CMA3652', u'CMA3662', u'CMA3682'],
      dtype='object', length=251)

In [11]:
sam_sum = sam.describe()
print_full(sam_sum.transpose())

                                           count  unique                  top  \
SNAPDATE                                  195343       1             20140905   
DWACCTID                                  195343    5119              4884754   
CUSTOMERNUMBER                            195343    5119           2360015969   
CYCLEDATE                                 195343     686           2014-08-15   
HADINCREASE                               195343       2                    0   
FIRSTMEASURABLEDATE                       195343      97           2014-09-09   
MAXREVIEWDATE                             192286      55  2014-09-03 00:00:00   
INCREASEDATE                               28503      83  2014-03-05 00:00:00   
INCRTYPE                                   28503       2            Temporary   
RANDOMNUM                                 195343    5119    0.000420096796202   
FIRSTSTATEMENTDATE                        195343    1111  1900-01-01 00:00:00   
FIRSTBSBCREDITORDERDATE     

In [ ]:
# all_col = sam.columns.str.upper()

In [12]:
# import math
# nan_col_ind = [math.isnan(val) if isinstance(val, float) else False for val in sam_sum.ix['top',]]
nan_col_ind = (sam_sum.ix['count',] == 0)
nan_col = sam.columns[nan_col_ind]
nan_col
# sam.drop(nan_col, axis=1, inplace=True)

Index([u'B5SCORE', u'B5RISKREASON1', u'B5RISKREASON2', u'B5RISKREASON3',
       u'B5RISKREASON4', u'VANTAGESCORE', u'VANTAGEREASON1', u'VANTAGEREASON2',
       u'VANTAGEREASON3', u'VANTAGEREASON4', u'OFACFLAG'],
      dtype='object')

In [13]:
oneval_col_ind = (sam_sum.ix['unique',] == 1)
oneval_col = sam.columns[oneval_col_ind]
oneval_col

Index([u'SNAPDATE', u'BRAND', u'LEGACYDAYSDELINQUENT', u'MOSTRECENTRECORDIND',
       u'CTCDCHARGEOFFSAMOUNT', u'LTCDCHARGEOFFSAMOUNT', u'MODELNAME'],
      dtype='object')

In [14]:
# import re
# [re.match('.*date.*', col, re.IGNORECASE) for col in all_col]
# matches = all_col.str.findall('.*date.*', re.IGNORECASE)
# matches.str.get(0).tolist()
date_col_ind = sam.columns.str.contains('DATE')
date_col = sam.columns[date_col_ind]
date_col

Index([u'SNAPDATE', u'CYCLEDATE', u'FIRSTMEASURABLEDATE', u'MAXREVIEWDATE',
       u'INCREASEDATE', u'FIRSTSTATEMENTDATE', u'FIRSTBSBCREDITORDERDATE',
       u'ACCOUNTOPENDATE', u'ACCOUNTCLOSEDATE', u'CHARGEOFFDATE',
       u'CLIINELIGIBLEDATE', u'LASTCOLLECTIONSCASEOPENDATE',
       u'LASTPAYMENTDATE', u'PAYMENTDUEDATE', u'REVIEWDATE',
       u'LASTBANKRUPTCYDATE'],
      dtype='object')

In [15]:
date_col2 = date_col.tolist()
date_col2.remove('CYCLEDATE')
date_col2

['SNAPDATE',
 'FIRSTMEASURABLEDATE',
 'MAXREVIEWDATE',
 'INCREASEDATE',
 'FIRSTSTATEMENTDATE',
 'FIRSTBSBCREDITORDERDATE',
 'ACCOUNTOPENDATE',
 'ACCOUNTCLOSEDATE',
 'CHARGEOFFDATE',
 'CLIINELIGIBLEDATE',
 'LASTCOLLECTIONSCASEOPENDATE',
 'LASTPAYMENTDATE',
 'PAYMENTDUEDATE',
 'REVIEWDATE',
 'LASTBANKRUPTCYDATE']

In [16]:
to_drop = nan_col.tolist() + oneval_col.tolist() + date_col2
sam2 = sam.drop(to_drop, axis=1)
sam2.shape

(195343, 219)

In [23]:
sam2['INCRTYPE'] = sam2['INCRTYPE'].fillna('Unknown')

In [31]:
sam2['CURRENTDAYSPASTDUE'].unique()

array(['0', '4', '2', '5', '7', '12'], dtype=object)

In [25]:
indata = sam2
out_data = pd.DataFrame([])
columns_in = sam2.columns.tolist()
columns_excl = ['CYCLEDATE', 'CUSTOMERSEGMENT', 'DWACCTID', 'CUSTOMERNUMBER', 'RANDOMNUM', 'CREDITCYCLEFACTMKEY']
all_cat = []
all_con = []
for col_name in columns_in:
    print col_name
    col_out = indata[col_name]
    if col_name not in columns_excl:
        if (sam_sum.ix['unique',col_name] > 20):
            out_data[col_name] = pd.to_numeric(col_out)
            all_con.append(col_name)
        if (sam_sum.ix['unique',col_name] <= 20):
            out_data[col_name] = pd.Series(pd.Categorical([v for v in col_out],categories=col_out.unique()))
            all_cat.append(col_name)
    else: out_data[col_name] = col_out

DWACCTID
CUSTOMERNUMBER
CYCLEDATE
HADINCREASE
INCRTYPE
RANDOMNUM
CREDITCYCLEFACTMKEY
SCORE01
SCORE02
SCORE03
SCORE04
SCORE05
SCORE06
SCORE07
SCORE08
SCORE09
STATEMENTMINIMUMPAYMENTDUE
ADDRESSBADIND
CURRENTBAL
ACCUMDEFERREDINTEREST
CLIINELIGIBLEIND
COLLECTIONCODE
CREDITLIMITAMOUNT
CREDITLIMITCHANGEREASON
CURRENTDAYSPASTDUE


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:15: FutureWarning: 
Setting NaNs in `categories` is deprecated and will be removed in a future version of pandas.


CURRENTDAYSPASTDUEDELINQUENT
CUSTOMERAPR
CUSTOMERSEGMENT
CYCLEDUE
DAYSDELQ
DAYSSINCELASTSALE
DECEASEDCODE
DTIFLAG
EMAILBADIND
EXTERNALSTATUSCODE
EXTERNALSTATUSREASONCODE
FINANCECHARGEBAL
HOMEPHONEBADIND
INTERNALSTATUSCODE
LTDMAXDAYSDELQ
LTDNETPAYMENTTALLY
MANUALSTATUSCODE
MARGINRATE
NONDEFERREDBAL
NOPAYDEFERREDBAL
NSFCODE
NUMBEROFMTHSINACTIVE
OTHERFEEBAL
OVERLIMITAMOUNT
PLACEDORDERFLAG
CTCDPAYMENTAMOUNT
CTCDPAYMENTRVSLAMOUNT
CTCDPAYMENTTALLY
CTCDBSBPURCHAMOUNT
CTCDBSBPURCHRVSLAMOUNT
CTCDBSBPURCHRVSLTALLY
CTCDBSBPURCHTALLY
CTCDNSFFEEAMOUNT
CTCDLATECHARGEAMOUNT
CTCDLATECHARGERVSLAMOUNT
LTCDPAYMENTAMOUNT
LTCDPAYMENTRVSLAMOUNT
LTCDPAYMENTTALLY
LTCDBSBPURCHAMOUNT
LTCDBSBPURCHRVSLAMOUNT
LTCDBSBPURCHRVSLTALLY
LTCDBSBPURCHTALLY
FHACCOUNTNUMBER
SCORE
FRAUDALERTFLAG
ADDRESSDISCREPANCYFLAG
FRAUDALERTCODE
CRITERIACODEANDDNSCODE
RISKREASON1
RISKREASON2
RISKREASON3
RISKREASON4
OPENITYPETRADEBALTOTAL
MO12BANKINSTALLCOUNT
OPENBANKRTYPEHIGHCREDITTOTAL
GOODRTYPETRADECOUNT
MO12REVOPENBANKCARDCOUNT
R6039_

In [32]:
out_data['CUSTOMERSEGMENT'] = pd.Series(pd.Categorical([v for v in out_data['CUSTOMERSEGMENT']],categories=out_data['CUSTOMERSEGMENT'].unique()))

In [49]:
out = out_data.sort_values(['CUSTOMERNUMBER', 'CYCLEDATE']).groupby(['CUSTOMERNUMBER']).last().reset_index()

In [100]:
con_ft_raw = out_data[all_con + ['CUSTOMERNUMBER']]
min_ft = con_ft_raw.groupby('CUSTOMERNUMBER', group_keys=False).apply(lambda x:x.min())
max_ft = con_ft_raw.groupby('CUSTOMERNUMBER').apply(lambda x:x.max())
con_ft = pd.merge(min_ft, max_ft, on='CUSTOMERNUMBER', suffixes=('_min', '_max'))

In [104]:
last_ft = out.drop(['DWACCTID', 'CYCLEDATE', 'RANDOMNUM', 'CREDITCYCLEFACTMKEY', 'CYCLEDUE'], axis=1)
last_ft['CUSTOMERNUMBER'] = last_ft['CUSTOMERNUMBER'].map(float)
all_ft = pd.merge(con_ft, last_ft, on='CUSTOMERNUMBER', suffixes=('', '_last'))

In [108]:
target_raw = out_data[['CUSTOMERNUMBER', 'CYCLEDUE']]
target = target_raw.groupby('CUSTOMERNUMBER').apply(lambda x:x.max())

In [111]:
dmat = pd.get_dummies(all_ft, prefix_sep='__')

In [115]:
dmat.drop(['CUSTOMERNUMBER'], axis=1, inplace=True)
target.drop(['CUSTOMERNUMBER'], axis=1, inplace=True)

In [116]:
xgb_model = xgb.XGBRegressor().fit(dmat, target)

In [123]:
wt = xgb_model.booster().get_score(importance_type='weight')

In [128]:
wt2 = pd.DataFrame(wt.items(), columns=['Feature', 'Weight'])
print_full(wt2.sort_values(['Weight'], ascending=False))

                                      Feature  Weight
108          CURRENTDAYSPASTDUEDELINQUENT_max     183
21                      DAYSSINCELASTSALE_max      37
107                               SCORE06_max      33
37                         LTDMAXDAYSDELQ_max      27
39                                SCORE01_min      19
82                   CTCDLATECHARGEAMOUNT_max      17
64                         NONDEFERREDBAL_max      16
26                            OTHERFEEBAL_max      15
34                   NUMBEROFMTHSINACTIVE_max      12
51                        OVERLIMITAMOUNT_max      11
50                                SCORE07_max      11
77                                SCORE02_max      10
74                        FHACCOUNTNUMBER_min      10
47                                CMA3856_max       9
73                          DAYSSINCELASTSALE       9
71                     LTDNETPAYMENTTALLY_max       9
93                       FINANCECHARGEBAL_max       9
13                 STATEMENT

In [131]:
dmat.shape

(5119, 813)

In [8]:
one_cust_id = sam_all['DWACCTID_x'].sample(1)
one_cust = sam_all.ix[sam_all['DWACCTID_x'].isin(one_cust_id),:]
print_full(one_cust)

      SNAPDATE_x DWACCTID_x CUSTOMERNUMBER_x CYCLEDATE_x HADINCREASE_x  \
46631   20140905    6118517       0005348708    20120919             1   
46632   20140905    6118517       0005348708    20121019             1   
46633   20140905    6118517       0005348708    20121119             1   
46634   20140905    6118517       0005348708    20121219             1   
46635   20140905    6118517       0005348708    20130119             1   
46636   20140905    6118517       0005348708    20130219             1   
46637   20140905    6118517       0005348708    20130319             1   
46638   20140905    6118517       0005348708    20130419             1   
46639   20140905    6118517       0005348708    20130519             1   
46640   20140905    6118517       0005348708    20130619             1   
46641   20140905    6118517       0005348708    20130719             1   
46642   20140905    6118517       0005348708    20130819             1   
46643   20140905    6118517       0005